In [252]:
import pandas as pd # vector junk
import json # json junk
import requests # handle requests
import folium # map rendering library
from bs4 import BeautifulSoup
import geopandas as gpd
from geopy.geocoders import Nominatim
print('Libraries imported.')

Libraries imported.


In [253]:
# Information created from the foursquare website to gain access to the api 
CLIENT_ID = 'X5K1NNIUOTZKXXKUNZYNGIOWPEK4WT4QU1PRDE2LEBOMMGF5'
CLIENT_SECRET = 'fsq3lMBZ7wsmrn8h8J1Rp/VHo5tAuxKiFoim2zid+FLT64g='
VERSION = 20220304
radius = 500
LIMIT = 100
print('Variables set') 

Variables set


In [254]:
# Crime data original source from folder
crime_orig = pd.read_csv("Neighbourhood_Crime_Rates.csv")
crime_orig.head()

,OBJECTID,Neighbourhood,Hood_ID,Population,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_2019,...,TheftOver_2015,TheftOver_2016,TheftOver_2017,TheftOver_2018,TheftOver_2019,TheftOver_AVG,TheftOver_CHG,TheftOver_Rate_2019,Shape__Area,Shape__Length
0,1,Yonge-St.Clair,97,12528,20,29,39,27,34,37,...,5,8,0,3,6,4.3,1.00,47.9,1.161315e+06,5873.270582
1,2,York University Heights,27,27593,271,296,361,344,357,370,...,46,37,39,38,28,36.3,-0.26,101.5,1.324666e+07,18504.777326
2,3,Lansing-Westgate,38,16164,44,80,68,85,75,72,...,5,5,11,6,11,7.0,0.83,68.1,5.346186e+06,11112.109625
3,4,Yorkdale-Glen Park,31,14804,106,136,174,161,175,209,...,14,26,23,20,29,22.5,0.45,195.9,6.038326e+06,10079.426920
4,5,Stonegate-Queensway,16,25051,88,71,76,95,87,82,...,8,4,6,7,4,6.0,-0.43,16.0,7.946202e+06,11853.189878


In [255]:
crime_df = crime_orig
# Assign variable name to list of relevant columns
crimes = ['Assault_2016', 'AutoTheft_2016', 'BreakandEnter_2016', 'Homicide_2016',
            'Robbery_2016', 'TheftOver_2016']
# Obtain Crime rate for each neighbourhood
crime_df['Crime_Rate'] = 100000 * (crime_df[crimes].sum(axis=1) /
                                     crime_df['Population'])
crime_df = pd.DataFrame(crime_df, columns=['Neighbourhood', 'Population', 'Assault_2016', 'AutoTheft_2016', 'BreakandEnter_2016', 'Homicide_2016',
            'Robbery_2016', 'TheftOver_2016', 'Crime_Rate'])
crime_df

,Neighbourhood,Population,Assault_2016,AutoTheft_2016,BreakandEnter_2016,Homicide_2016,Robbery_2016,TheftOver_2016,Crime_Rate
0,Yonge-St.Clair,12528,39,7,12,0,6,8,574.712644
1,York University Heights,27593,361,105,98,2,70,37,2439.024390
2,Lansing-Westgate,16164,68,27,41,0,9,5,927.988122
3,Yorkdale-Glen Park,14804,174,41,66,1,24,26,2242.637125
4,Stonegate-Queensway,25051,76,12,49,0,16,4,626.721488
...,...,...,...,...,...,...,...,...,...
135,Milliken,26572,88,35,67,0,40,10,903.206383
136,Pleasant View,15818,58,8,32,0,13,3,720.697939
137,Wychwood,14349,86,18,29,0,9,4,1017.492508
138,Leaside-Bennington,16828,33,4,32,0,8,2,469.455669


In [256]:
# Neighbourhood Data

info_df_orig = pd.read_json('neighbourhood-profiles-2016-csv.json')
# info_df_orig.rename(columns={'Neighbourhood':'Characteristic'})
# print(info_df_orig.columns)
info_df = info_df_orig.set_index('Characteristic').T

info_df


Characteristic,Neighbourhood Number,TSNS2020 Designation,"Population, 2016","Population, 2011",Population Change 2011-2016,Total private dwellings,Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Children (0-14 years),...,Intraprovincial migrants,Interprovincial migrants,External migrants,Total - Mobility status 5 years ago - 25% sample data,Non-movers,Movers,Non-migrants,Migrants,Interprovincial migrants,External migrants
Agincourt North,129,No Designation,"29,113","30,279",-3.90%,"9,371","9,120","3,929",7.41,"3,840",...,275,75,605,"27,490","18,865","8,610","5,445","3,170",135,"2,280"
Agincourt South-Malvern West,128,No Designation,"23,757","21,988",8.00%,"8,535","8,136","3,034",7.83,"3,075",...,320,90,490,"22,325","13,565","8,775","5,610","3,145",220,"2,170"
Alderwood,20,No Designation,"12,054","11,904",1.30%,"4,732","4,616","2,435",4.95,"1,760",...,220,40,70,"11,370","8,235","3,130","2,200",925,70,245
Annex,95,No Designation,"30,526","29,177",4.60%,"18,109","15,934","10,863",2.81,"2,360",...,900,385,835,"27,715","12,980","14,735","8,340","6,390","1,310","2,460"
Banbury-Don Mills,42,No Designation,"27,695","26,918",2.90%,"12,473","12,124","2,775",9.98,"3,605",...,345,75,380,"25,925","16,300","9,625","6,480","3,140",220,"1,735"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yonge-Eglinton,100,No Designation,"11,817","10,578",11.70%,"6,103","5,676","7,162",1.65,"1,800",...,345,130,325,"11,230","5,785","5,450","3,020","2,425",290,"1,160"
Yonge-St.Clair,97,No Designation,"12,528","11,652",7.50%,"7,475","7,012","10,708",1.17,"1,210",...,320,110,265,"11,865","5,660","6,195","3,895","2,310",325,955
York University Heights,27,NIA,"27,593","27,713",-0.40%,"11,051","10,170","2,086",13.23,"4,045",...,500,180,680,"26,110","14,720","11,400","6,435","4,965",195,"3,285"
Yorkdale-Glen Park,31,Emerging Neighbourhood,"14,804","14,687",0.80%,"5,847","5,344","2,451",6.04,"1,960",...,145,35,140,"13,420","9,125","4,290","2,940","1,345",135,775


In [257]:
merged = pd.merge(crime_df,info_df,left_on=['Neighbourhood'],right_index=True)
merged

,Neighbourhood,Population,Assault_2016,AutoTheft_2016,BreakandEnter_2016,Homicide_2016,Robbery_2016,TheftOver_2016,Crime_Rate,Neighbourhood Number,...,Intraprovincial migrants,Interprovincial migrants,External migrants,Total - Mobility status 5 years ago - 25% sample data,Non-movers,Movers,Non-migrants,Migrants,Interprovincial migrants,External migrants
0,Yonge-St.Clair,12528,39,7,12,0,6,8,574.712644,97,...,320,110,265,"11,865","5,660","6,195","3,895","2,310",325,955
1,York University Heights,27593,361,105,98,2,70,37,2439.024390,27,...,500,180,680,"26,110","14,720","11,400","6,435","4,965",195,"3,285"
2,Lansing-Westgate,16164,68,27,41,0,9,5,927.988122,38,...,440,110,510,"15,275","7,930","7,335","4,170","3,165",220,"1,965"
3,Yorkdale-Glen Park,14804,174,41,66,1,24,26,2242.637125,31,...,145,35,140,"13,420","9,125","4,290","2,940","1,345",135,775
4,Stonegate-Queensway,25051,76,12,49,0,16,4,626.721488,16,...,480,150,225,"23,510","16,000","7,515","4,780","2,730",250,"1,245"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Milliken,26572,88,35,67,0,40,10,903.206383,130,...,290,10,500,"24,730","16,725","8,000","5,105","2,895",85,"2,120"
136,Pleasant View,15818,58,8,32,0,13,3,720.697939,46,...,250,75,575,"15,255","9,910","5,345","2,865","2,475",110,"1,765"
137,Wychwood,14349,86,18,29,0,9,4,1017.492508,94,...,140,60,225,"12,630","7,930","4,700","3,355","1,350",210,575
138,Leaside-Bennington,16828,33,4,32,0,8,2,469.455669,56,...,125,85,190,"15,845","11,135","4,710","3,560","1,145",200,380


In [258]:
#list of postal codes and neighborhood names with district extracted from url page
postalCode=[]
neighborhood=[]

#Website to extract the data from
url = "https://www.zipcodesonline.com/2020/06/postal-code-of-toronto-in-2020.html"

#Read the html tables into a list
scrape = pd.read_html(url,skiprows=2)

#Store the date into the list 
neighborhood = scrape[0][1]
postalCode = scrape[0][2]

#Check data 
#print(neighborhood)
#print(postalCode)


#Create a data from scraped data
PN = pd.DataFrame({'Postal Code':postalCode,'Neighbourhood':neighborhood})
PN.columns = ['Postal Code', 'Neighbourhood']

PN
#postalCodeLongLat = pd.read_csv('Toronto_LocationData.csv')
#postalCodeLongLat



,Postal Code,Neighbourhood
0,M5H,Adelaide
1,M1V,Agincourt North
2,M1S,Agincourt
3,M9V,Albion Gardens
4,M8W,Alderwood
...,...,...
200,M4B,"Woodbine Gardens,"
201,M4C,Woodbine Heights
202,M2P,York Mills West
203,M2L,York Mills


In [259]:
postalCodeLongLat = pd.read_csv('Toronto_LocationData.csv')
postalCodeLongLat

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [260]:
MergeLocation = pd.merge(postalCodeLongLat, PN, on= "Postal Code")
MergeLocation.rename(columns={'Postal Code':'Postal Code', 'Postal Code':'Postal Code', 'Neighbourhood': 'Neighbourhood', 'Latitude': 'Latitude', 'Longitude': 'Longitude'})
MergeLocation


,Postal Code,Latitude,Longitude,Neighbourhood
0,M1B,43.806686,-79.194353,Malvern
1,M1B,43.806686,-79.194353,Rouge
2,M1C,43.784535,-79.160497,Highland Creek
3,M1C,43.784535,-79.160497,Port Union
4,M1C,43.784535,-79.160497,Rouge Hill
...,...,...,...,...
197,M9V,43.739416,-79.588437,Silverstone
198,M9V,43.739416,-79.588437,South Steeles
199,M9V,43.739416,-79.588437,Thistletown
200,M9W,43.706748,-79.594054,Northwest


In [261]:
#Get Latitude and Longitude from Neighbourhoods File

#Read the shape file to get longitude and latitude when crs epsg=4326
LocationData = gpd.read_file("Neighbourhoods/Neighbourhoods.shp")

#Converted to get Latitude and Longitude
LocationData = LocationData.to_crs(epsg=4326)

#Enter data into a dataframe
NeighbourhoodLocationData = pd.DataFrame(LocationData,columns=['FIELD_7', 'FIELD_12', 'FIELD_11', 'geometry'])
#NeighbourhoodLocationData = pd.DataFrame(LocationData)
NeighbourhoodLocationData
# #Rename all columns
NeighbourhoodLocationData.rename(columns={'FIELD_7':'Neighbourhood', 'FIELD_12':'Latitude', 'FIELD_11':'Longitude', 'geometry':'geometry'}, inplace=True)

# #Get all the values before the bracket
NeighbourhoodLocationData['Neighbourhood'] = NeighbourhoodLocationData["Neighbourhood"].astype(str)
NeighbourhoodLocationData['Neighbourhood'] = NeighbourhoodLocationData['Neighbourhood'].str.split(" \(", 1)
NeighbourhoodLocationData['Neighbourhood'] = NeighbourhoodLocationData['Neighbourhood'].str.get(0)

NeighbourhoodLocationData.head()
#print(NeighbourhoodLocationData['Neighbourhood'].value_counts())
#NeighbourhoodLocationData.to_csv("outputL.csv")

C:\Users\moham\anaconda3\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Neighbourhood,Latitude,Longitude,geometry
0,Wychwood,43.676919,-79.425515,"POLYGON ((-79.43592 43.68015, -79.43492 43.680..."
1,Yonge-Eglinton,43.704689,-79.403590,"POLYGON ((-79.41096 43.70408, -79.40962 43.704..."
2,Yonge-St.Clair,43.687859,-79.397871,"POLYGON ((-79.39119 43.68108, -79.39141 43.680..."
3,York University Heights,43.765736,-79.488883,"POLYGON ((-79.50529 43.75987, -79.50488 43.759..."
4,Yorkdale-Glen Park,43.714672,-79.457108,"POLYGON ((-79.43969 43.70561, -79.44011 43.705..."


In [262]:
#combination of all the datasets
#prints to check columns in each dataframe

# print(NeighbourhoodLocationData.columns)
# print(MergeLocation.columns)
# print(merged.columns)

MergeLocation = pd.DataFrame(MergeLocation, columns= ['Postal Code', 'Neighbourhood'])
#Combination of postal code data and neighbourhood information

#Step 1 merge information on Neighbourhood location data and Neighbourhood profiles
F1 = pd.merge(merged, NeighbourhoodLocationData, on="Neighbourhood")
# F1

#F1.to_csv("outputF1.csv")


#Step 2 merge information on Neighbourhood postal code data and previous merge
# F2 = pd.merge(F1, MergeLocation, on="Neighbourhood")
# F2
F2 = F1[['Neighbourhood', 'Population','Crime_Rate', 'Youth (15-24 years)', 'Working Age (25-54 years)', 'Latitude', 'Longitude', 'geometry']]
F2


,Neighbourhood,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Latitude,Longitude,geometry
0,Yonge-St.Clair,12528,574.712644,920,"5,960",43.687859,-79.397871,"POLYGON ((-79.39119 43.68108, -79.39141 43.680..."
1,York University Heights,27593,2439.024390,"4,750","12,290",43.765736,-79.488883,"POLYGON ((-79.50529 43.75987, -79.50488 43.759..."
2,Lansing-Westgate,16164,927.988122,"2,105","7,590",43.754271,-79.424748,"POLYGON ((-79.43998 43.76156, -79.44004 43.761..."
3,Yorkdale-Glen Park,14804,2242.637125,"1,870","5,860",43.714672,-79.457108,"POLYGON ((-79.43969 43.70561, -79.44011 43.705..."
4,Stonegate-Queensway,25051,626.721488,"2,495","10,660",43.635518,-79.501128,"POLYGON ((-79.49262 43.64744, -79.49277 43.647..."
...,...,...,...,...,...,...,...,...
131,Milliken,26572,903.206383,"3,195","10,565",43.820691,-79.275009,"POLYGON ((-79.24308 43.81297, -79.24433 43.812..."
132,Pleasant View,15818,720.697939,"2,080","6,470",43.786982,-79.334948,"POLYGON ((-79.34346 43.79517, -79.34359 43.795..."
133,Wychwood,14349,1017.492508,"1,320","6,420",43.676919,-79.425515,"POLYGON ((-79.43592 43.68015, -79.43492 43.680..."
134,Leaside-Bennington,16828,469.455669,"2,175","6,455",43.703797,-79.366072,"POLYGON ((-79.37749 43.71309, -79.37762 43.713..."


In [263]:
radius = 2000
limit = 50
category = 13064


places = []

for lat, long, neighbourhood in zip(NeighbourhoodLocationData['Latitude'],
                                   NeighbourhoodLocationData['Longitude'],
                                   NeighbourhoodLocationData['Neighbourhood']):
    
    url = "https://api.foursquare.com/v3/places/search?ll={}%2C{}&radius={}&categories={}&limit={}".format(lat,long,radius,category,limit)
   
    headers = {

        "Accept": "application/json",

        "Authorization": "fsq346wopDU+Ro11RNJhAnUwpiz/2xgtt3pjAunkFmd1c+8="

    }


    response = requests.request("GET", url, headers=headers).json()["results"]
    
                                                                    
    for place in response:
        places.append((neighbourhood,
                      lat,
                      long,
                      place['name'],
                      place['geocodes']['main']['latitude'],
                      place['geocodes']['main']['longitude'],
                     ))
        
print("Loaded to list!")


Loaded to list!


In [264]:
places_df = pd.DataFrame(places)
# pd.set_option("display.max_rows", None)

places_df.columns = ['Neighbourhood', 'Latitude', 'Longitude', 'Name', 'VenueLatitude', 'VenueLongitude']
places_df



,Neighbourhood,Latitude,Longitude,Name,VenueLatitude,VenueLongitude
0,Wychwood,43.676919,-79.425515,Ferro Bar & Cafe,43.680943,-79.428443
1,Wychwood,43.676919,-79.425515,Marcello's Pizzeria,43.677841,-79.442494
2,Wychwood,43.676919,-79.425515,Churrasco of St Clair,43.681630,-79.425185
3,Wychwood,43.676919,-79.425515,Napoli Centrale,43.669377,-79.413027
4,Wychwood,43.676919,-79.425515,Pizza e Pazzi,43.677954,-79.444009
...,...,...,...,...,...,...
3751,L'Amoreaux,43.795716,-79.314084,Chef's Pizza Fish & Chips,43.795980,-79.327180
3752,L'Amoreaux,43.795716,-79.314084,Angela Pizza & Wings,43.787750,-79.329066
3753,L'Amoreaux,43.795716,-79.314084,Sammy's Pizza & Restaurant,43.786489,-79.328632
3754,L'Amoreaux,43.795716,-79.314084,Pizza 3 Toppings,43.783493,-79.299138


In [265]:
# pd.set_option("display.max_rows", None)

places_df
count = pd.DataFrame(places_df['Neighbourhood'].value_counts().reset_index())
count = count.rename(columns={'index':'Neighbourhood', 'Neighbourhood':'Total Pizzerias'})
count



,Neighbourhood,Total Pizzerias
0,Wychwood,50
1,Trinity-Bellwoods,50
2,North St.James Town,50
3,Oakwood Village,50
4,Palmerston-Little Italy,50
...,...,...
135,Edenbridge-Humber Valley,4
136,Bayview Village,3
137,Princess-Rosethorn,3
138,Bayview Woods-Steeles,2


In [266]:
#Frame of all relevent information plus venue information
F3 = pd.merge(F2, count, on="Neighbourhood")
F3

,Neighbourhood,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Latitude,Longitude,geometry,Total Pizzerias
0,Yonge-St.Clair,12528,574.712644,920,"5,960",43.687859,-79.397871,"POLYGON ((-79.39119 43.68108, -79.39141 43.680...",30
1,York University Heights,27593,2439.024390,"4,750","12,290",43.765736,-79.488883,"POLYGON ((-79.50529 43.75987, -79.50488 43.759...",21
2,Lansing-Westgate,16164,927.988122,"2,105","7,590",43.754271,-79.424748,"POLYGON ((-79.43998 43.76156, -79.44004 43.761...",18
3,Yorkdale-Glen Park,14804,2242.637125,"1,870","5,860",43.714672,-79.457108,"POLYGON ((-79.43969 43.70561, -79.44011 43.705...",26
4,Stonegate-Queensway,25051,626.721488,"2,495","10,660",43.635518,-79.501128,"POLYGON ((-79.49262 43.64744, -79.49277 43.647...",17
...,...,...,...,...,...,...,...,...,...
131,Milliken,26572,903.206383,"3,195","10,565",43.820691,-79.275009,"POLYGON ((-79.24308 43.81297, -79.24433 43.812...",11
132,Pleasant View,15818,720.697939,"2,080","6,470",43.786982,-79.334948,"POLYGON ((-79.34346 43.79517, -79.34359 43.795...",22
133,Wychwood,14349,1017.492508,"1,320","6,420",43.676919,-79.425515,"POLYGON ((-79.43592 43.68015, -79.43492 43.680...",50
134,Leaside-Bennington,16828,469.455669,"2,175","6,455",43.703797,-79.366072,"POLYGON ((-79.37749 43.71309, -79.37762 43.713...",24


In [267]:
Ftemp = pd.DataFrame(F3, columns=['geometry'])

In [268]:
#Dataset without commas in ints
F4 = pd.DataFrame(F3,columns=['Neighbourhood', 'Population','Crime_Rate', 'Youth (15-24 years)', 'Working Age (25-54 years)', 'Total Pizzerias', 'Latitude', 'Longitude'])

F4['Youth (15-24 years)'] = pd.to_numeric(F3['Youth (15-24 years)'].str.replace(",",""))
F4['Working Age (25-54 years)'] = pd.to_numeric(F3['Working Age (25-54 years)'].str.replace(",",""))

F4

,Neighbourhood,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Total Pizzerias,Latitude,Longitude
0,Yonge-St.Clair,12528,574.712644,920,5960,30,43.687859,-79.397871
1,York University Heights,27593,2439.024390,4750,12290,21,43.765736,-79.488883
2,Lansing-Westgate,16164,927.988122,2105,7590,18,43.754271,-79.424748
3,Yorkdale-Glen Park,14804,2242.637125,1870,5860,26,43.714672,-79.457108
4,Stonegate-Queensway,25051,626.721488,2495,10660,17,43.635518,-79.501128
...,...,...,...,...,...,...,...,...
131,Milliken,26572,903.206383,3195,10565,11,43.820691,-79.275009
132,Pleasant View,15818,720.697939,2080,6470,22,43.786982,-79.334948
133,Wychwood,14349,1017.492508,1320,6420,50,43.676919,-79.425515
134,Leaside-Bennington,16828,469.455669,2175,6455,24,43.703797,-79.366072


In [269]:
from sklearn.cluster import KMeans
Cluster_data=F4.copy(deep=True) # make a new dataframe for clustering
# Drop columns not required for the clustering algorithm
ClusteringPer = Cluster_data.drop(labels=['Neighbourhood'], axis=1)

# set number of clusters
k = 4

# run k-means clustering
kmean_algo = KMeans(n_clusters=k, random_state=0).fit(ClusteringPer)

# check cluster labels generated for each row in the dataframe
kmean_algo.labels_[0:7] 

# add clustering labels to original dataframe
Cluster_data.insert(1, 'Clusters', kmean_algo.labels_)
#Cluster_data.to_csv('CleanData/Clusters.csv')
# print(kmeans.labels_)

Cluster_data = pd.concat([Cluster_data, Ftemp], join='inner', axis=1)
Cluster_data

,Neighbourhood,Clusters,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Total Pizzerias,Latitude,Longitude,geometry
0,Yonge-St.Clair,3,12528,574.712644,920,5960,30,43.687859,-79.397871,"POLYGON ((-79.39119 43.68108, -79.39141 43.680..."
1,York University Heights,2,27593,2439.024390,4750,12290,21,43.765736,-79.488883,"POLYGON ((-79.50529 43.75987, -79.50488 43.759..."
2,Lansing-Westgate,0,16164,927.988122,2105,7590,18,43.754271,-79.424748,"POLYGON ((-79.43998 43.76156, -79.44004 43.761..."
3,Yorkdale-Glen Park,3,14804,2242.637125,1870,5860,26,43.714672,-79.457108,"POLYGON ((-79.43969 43.70561, -79.44011 43.705..."
4,Stonegate-Queensway,2,25051,626.721488,2495,10660,17,43.635518,-79.501128,"POLYGON ((-79.49262 43.64744, -79.49277 43.647..."
...,...,...,...,...,...,...,...,...,...,...
131,Milliken,2,26572,903.206383,3195,10565,11,43.820691,-79.275009,"POLYGON ((-79.24308 43.81297, -79.24433 43.812..."
132,Pleasant View,0,15818,720.697939,2080,6470,22,43.786982,-79.334948,"POLYGON ((-79.34346 43.79517, -79.34359 43.795..."
133,Wychwood,3,14349,1017.492508,1320,6420,50,43.676919,-79.425515,"POLYGON ((-79.43592 43.68015, -79.43492 43.680..."
134,Leaside-Bennington,0,16828,469.455669,2175,6455,24,43.703797,-79.366072,"POLYGON ((-79.37749 43.71309, -79.37762 43.713..."


In [270]:
averageOfClusters = Cluster_data.groupby('Clusters').mean()
averageOfClusters = pd.DataFrame(averageOfClusters, columns=['Population', 'Crime_Rate', 'Youth (15-24 years)', 'Working Age (25-54 years)', 'Total Pizzerias'])
averageOfClusters

,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Total Pizzerias
Clusters,,,,,
0,19216.693878,1151.316992,2356.530612,8410.204082,24.265306
1,49725.714286,1016.712601,6597.857143,23875.000000,19.714286
2,29346.680000,1467.193601,3871.600000,13465.800000,24.080000
3,11506.545455,1058.076841,1348.363636,4994.000000,30.127273


In [271]:
# Cluster 1 seems like the best fit when examining the business case

In [272]:
# Assigning the clusters to their own dataframes to eventually plot
#Cluster_data
is_0 = Cluster_data['Clusters'] == 0
Cluster0 = Cluster_data[is_0]

is_1 = Cluster_data['Clusters'] == 1
Cluster1 = Cluster_data[is_1]

is_2 = Cluster_data['Clusters'] == 2
Cluster2 = Cluster_data[is_2]

is_3 = Cluster_data['Clusters'] == 3
Cluster3 = Cluster_data[is_3]

In [273]:
#Cluster0 ## 2nd ranked data set

In [274]:
#Cluster1 ## Best Dataset 1st ranked data set

In [275]:
#Cluster2 ## 3rd ranked data set

In [276]:
#Cluster3 ## worst data set 4th ranked data set

In [277]:
#Adding filling on the map for locations

map = folium.Map(location=[43.65, - 79.38], zoom_start=100)
# adding colour of gold
for x, r in Cluster0.iterrows():
    simulation = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geometryinfo = simulation.to_json()
    geometryinfo = folium.GeoJson(data=geometryinfo,
                           style_function=lambda x: {'fillColor': 'Yellow'})
    folium.Popup(r['Neighbourhood']).add_to(geometryinfo)
    geometryinfo.add_to(map)

for x, r in Cluster1.iterrows():
    simulation = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geometryinfo = simulation.to_json()
    geometryinfo = folium.GeoJson(data=geometryinfo,
                           style_function=lambda x: {'fillColor': 'Green'})
    folium.Popup(r['Neighbourhood']).add_to(geometryinfo)
    geometryinfo.add_to(map)

for x, r in Cluster2.iterrows():
    simulation = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geometryinfo = simulation.to_json()
    geometryinfo = folium.GeoJson(data=geometryinfo,
                           style_function=lambda x: {'fillColor': 'Cyan'})
    folium.Popup(r['Neighbourhood']).add_to(geometryinfo)
    geometryinfo.add_to(map)

for x, r in Cluster3.iterrows():
    simulation = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geometryinfo = simulation.to_json()
    geometryinfo = folium.GeoJson(data=geometryinfo,
                           style_function=lambda x: {'fillColor': 'Red'})
    folium.Popup(r['Neighbourhood']).add_to(geometryinfo)
    geometryinfo.add_to(map)

map

In [278]:
# Best data set represents green on the folium map above
Cluster1

,Neighbourhood,Clusters,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Total Pizzerias,Latitude,Longitude,geometry
11,Islington-City Centre West,1,43965,1221.426134,4695,20640,18,43.633463,-79.543317,"POLYGON ((-79.51317 43.62049, -79.51505 43.620..."
39,Waterfront Communities-The Island,1,65913,1922.230819,7840,45105,50,43.633880,-79.377202,"POLYGON ((-79.37697 43.64688, -79.37576 43.647..."
82,L'Amoreaux,1,43993,656.922692,5730,17210,11,43.795716,-79.314084,"POLYGON ((-79.28857 43.79607, -79.28921 43.795..."
93,Willowdale East,1,50434,616.647500,6940,25850,29,43.770602,-79.401484,"POLYGON ((-79.41225 43.76669, -79.41232 43.766..."
95,Rouge,1,46496,612.955953,6700,18510,2,43.821201,-79.186343,"POLYGON ((-79.19701 43.79652, -79.19705 43.796..."
108,Woburn,1,53485,1043.283164,7660,21945,20,43.766740,-79.228586,"POLYGON ((-79.21128 43.75113, -79.21135 43.751..."
119,Malvern,1,43794,1043.521944,6620,17865,8,43.803658,-79.222517,"POLYGON ((-79.19701 43.79652, -79.19744 43.796..."


In [286]:
Final_Params = Cluster1
Final_Params['Ideal Factor'] = (Cluster1['Youth (15-24 years)']+Cluster1['Working Age (25-54 years)'])/(Cluster1['Total Pizzerias'] * Cluster1['Crime_Rate'])

best_value = Final_Params['Ideal Factor'].max()

is_best = Cluster1['Ideal Factor'] == best_value
Prime_Location = Cluster1[is_best]
Prime_Location

C:\Users\moham\AppData\Local\Temp/ipykernel_8576/1609933852.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Params['Ideal Factor'] = (Cluster1['Youth (15-24 years)']+Cluster1['Working Age (25-54 years)'])/(Cluster1['Total Pizzerias'] * Cluster1['Crime_Rate'])


,Neighbourhood,Clusters,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Total Pizzerias,Latitude,Longitude,geometry,Ideal Factor
95,Rouge,1,46496,612.955953,6700,18510,2,43.821201,-79.186343,"POLYGON ((-79.19701 43.79652, -79.19705 43.796...",20.564284
